## Basis Swap Par Spreads

The goal of the project is to compute the par spreads of a pool of basis swaps, i.e. those contracts in which two floating libor rates, with different tenors, are exchanged. The par spread is that margin which has to be paid/received on top of the floating rate with the shorter tenor.

The input parameters are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotation)
* 3 sets of forward libor rates and their fixing dates, one for the Libor 1M, one for the Libor 3M and one for the Libor 6M.
* The static data of the pool of swaps (nominal, maturity, tenor of the first leg, tenor of the second leg)

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for multi-curve evaluation.

### Notes and hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for evaluation of risky flows.

All maturities are expressed in months and rates are expressed as fractions of one (i.e. 0.01 means 1%).

Remember to reuse the code we developed during the lessons in `finmarkets.py` as much as possible, instead of trying to rewrite everything from scratch!

In [1]:
from finmarkets import DiscountCurve, OvernightIndexSwap, generate_swap_dates, ForwardRateCurve
from datetime import date

today = date(2019, 10, 31)

ois_quotes = [
    {'maturity': 1, 'rate': 0.00106},
    {'maturity': 2, 'rate': 0.00114},
    {'maturity': 3, 'rate': 0.00115},
    {'maturity': 4, 'rate': 0.00117},
    {'maturity': 5, 'rate': 0.00119},
    {'maturity': 6, 'rate': 0.00121},
    {'maturity': 7, 'rate': 0.00122},
    {'maturity': 8, 'rate': 0.00124},
    {'maturity': 9, 'rate': 0.00128},
    {'maturity': 10, 'rate': 0.00131},
    {'maturity': 11, 'rate': 0.00135},
    {'maturity': 12, 'rate': 0.00138},
    {'maturity': 15, 'rate': 0.00152},
    {'maturity': 18, 'rate': 0.00166},
    {'maturity': 21, 'rate': 0.00184},
    {'maturity': 24, 'rate': 0.00206},
    {'maturity': 36, 'rate': 0.00344},
    {'maturity': 48, 'rate': 0.00543},
    {'maturity': 60, 'rate': 0.00756},
    {'maturity': 72, 'rate': 0.00967},
    {'maturity': 84, 'rate': 0.01162},
    {'maturity': 96, 'rate': 0.0134},
    {'maturity': 108, 'rate': 0.01502},
    {'maturity': 120, 'rate': 0.01649},
    {'maturity': 132, 'rate': 0.01776},
    {'maturity': 144, 'rate': 0.01888},
    {'maturity': 180, 'rate': 0.02137},
    {'maturity': 240, 'rate': 0.02322},
    {'maturity': 300, 'rate': 0.02389},
    {'maturity': 360, 'rate': 0.02416},
]

euribor_1m_curve = ForwardRateCurve(
    [
        date(2019, 10, 31), 
        date(2020, 4, 30),
        date(2020, 10, 30), 
        date(2021, 4, 30), 
        date(2021, 10, 30), 
        date(2022, 4, 30), 
        date(2022, 10, 30), 
        date(2023, 4, 30), 
        date(2023, 10, 30), 
        date(2024, 4, 30), 
        date(2024, 10, 30),
        date(2025, 4, 30),
        date(2025, 10, 30),
        date(2026, 4, 30),
        date(2026, 10, 30),
        date(2027, 4, 30), 
        date(2027, 10, 30), 
        date(2028, 4, 30),
        date(2028, 10, 30),
        date(2029, 4, 30), 
        date(2029, 10, 30), 
        date(2030, 4, 30), 
        date(2030, 10, 30),
        date(2031, 4, 30), 
        date(2031, 10, 30),
        date(2032, 4, 30),
        date(2032, 10, 30), 
        date(2033, 4, 30), 
        date(2033, 10, 30), 
        date(2034, 4, 30), 
    ],
    [
        0.002,
        0.00431709159584514,
        0.00513437204910293,
        0.00595155807365439,
        0.00676883852691218,
        0.00758611898016997,
        0.00840339943342776,
        0.00922058545797923,
        0.010037865911237,
        0.0108550519357885,
        0.0116723323890463,
        0.0124895184135977,
        0.0133067988668555,
        0.0141240793201133,
        0.0149413597733711,
        0.0157585457979226,
        0.0165758262511804,
        0.0173930122757318,
        0.0182102927289896,
        0.0190274787535411,
        0.0198447592067989,
        0.0206620396600567,
        0.0214793201133144,
        0.0222965061378659,
        0.0231137865911237,
        0.0239309726156752,
        0.024748253068933,
        0.0255654390934844,
        0.0263827195467422,
        0.0267,
    ]
)

euribor_3m_curve = ForwardRateCurve(
    [
        date(2019, 10, 31), 
        date(2020, 4, 30),
        date(2020, 10, 30), 
        date(2021, 4, 30), 
        date(2021, 10, 30), 
        date(2022, 4, 30), 
        date(2022, 10, 30), 
        date(2023, 4, 30), 
        date(2023, 10, 30), 
        date(2024, 4, 30), 
        date(2024, 10, 30),
        date(2025, 4, 30),
        date(2025, 10, 30),
        date(2026, 4, 30),
        date(2026, 10, 30),
        date(2027, 4, 30), 
        date(2027, 10, 30), 
        date(2028, 4, 30),
        date(2028, 10, 30),
        date(2029, 4, 30), 
        date(2029, 10, 30), 
        date(2030, 4, 30), 
        date(2030, 10, 30),
        date(2031, 4, 30), 
        date(2031, 10, 30),
        date(2032, 4, 30),
        date(2032, 10, 30), 
        date(2033, 4, 30), 
        date(2033, 10, 30), 
        date(2034, 4, 30), 
    ],
    [
        0.0035,
        0.00431709159584514,
        0.00513437204910293,
        0.00595155807365439,
        0.00676883852691218,
        0.00758611898016997,
        0.00840339943342776,
        0.00922058545797923,
        0.010037865911237,
        0.0108550519357885,
        0.0116723323890463,
        0.0124895184135977,
        0.0133067988668555,
        0.0141240793201133,
        0.0149413597733711,
        0.0157585457979226,
        0.0165758262511804,
        0.0173930122757318,
        0.0182102927289896,
        0.0190274787535411,
        0.0198447592067989,
        0.0206620396600567,
        0.0214793201133144,
        0.0222965061378659,
        0.0231137865911237,
        0.0239309726156752,
        0.024748253068933,
        0.0255654390934844,
        0.0263827195467422,
        0.0272,
    ]
)

euribor_6m_curve = ForwardRateCurve(
    [
        date(2019, 10, 31), 
        date(2020, 4, 30),
        date(2020, 10, 30), 
        date(2021, 4, 30), 
        date(2021, 10, 30), 
        date(2022, 4, 30), 
        date(2022, 10, 30), 
        date(2023, 4, 30), 
        date(2023, 10, 30), 
        date(2024, 4, 30), 
        date(2024, 10, 30),
        date(2025, 4, 30),
        date(2025, 10, 30),
        date(2026, 4, 30),
        date(2026, 10, 30),
        date(2027, 4, 30), 
        date(2027, 10, 30), 
        date(2028, 4, 30),
        date(2028, 10, 30),
        date(2029, 4, 30), 
        date(2029, 10, 30), 
        date(2030, 4, 30), 
        date(2030, 10, 30),
        date(2031, 4, 30), 
        date(2031, 10, 30),
        date(2032, 4, 30),
        date(2032, 10, 30), 
        date(2033, 4, 30), 
        date(2033, 10, 30), 
        date(2034, 4, 30), 
    ],
    [
        0.005,
        0.0058,
        0.0066,
        0.0074,
        0.0082,
        0.009,
        0.0098,
        0.0106,
        0.0114,
        0.0122,
        0.013,
        0.0138,
        0.0146,
        0.0154,
        0.0162,
        0.017,
        0.0178,
        0.0186,
        0.0194,
        0.0202,
        0.021,
        0.0218,
        0.0226,
        0.0234,
        0.0242,
        0.025,
        0.0258,
        0.0266,
        0.0274,
        0.0282,
    ]
)

basis_swaps = [
    {'nominal': 1000000, 'maturity': 12, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 12, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 12, 'first_tenor': 1, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 24, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 24, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 24, 'first_tenor': 1, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 60, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 60, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 60, 'first_tenor': 1, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 90, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 90, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 90, 'first_tenor': 1, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 120, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 120, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 120, 'first_tenor': 1, 'second_tenor': 6},
]

First of all we need to determine the discount curve from the OIS market quotes using the bootstrap technique.

* generate set of OIS according to the market quotes
* create an objective function with a discount curve and the squared sum of NPVs
* check minimization results
* create the found `DiscountCurve`

In [2]:
from finmarkets import generate_swap_dates, OvernightIndexSwap
pillar_dates = [today]

swaps = []
for quote in ois_quotes:
    swap = OvernightIndexSwap(
           1e6, 
           generate_swap_dates(today, quote['maturity']),
           0.01 * quote['rate'])
    swaps.append(swap)
    pillar_dates.append(swap.payment_dates[-1])
    
pillar_dates = sorted(pillar_dates)

In [3]:
from finmarkets import DiscountCurve

def objective_function(x):
    curve = DiscountCurve(today, pillar_dates, x)
    
    sum_sq = 0.0
    for swap in swaps:
        sum_sq += swap.npv(curve) ** 2
        
    return sum_sq

In [4]:
from scipy.optimize import minimize

x0 = [1.0 for i in range(len(pillar_dates))]
bounds = [(0.01, 100.0) for i in range(len(pillar_dates))]
bounds[0] = (1.0, 1.0)

result = minimize(objective_function, x0, bounds=bounds)

In [5]:
print (result)
print (objective_function(x0))
print (objective_function(result.x))

      fun: 0.0007487918883500543
 hess_inv: <31x31 LbfgsInvHessProduct with dtype=float64>
      jac: array([  5.99758142e+05,   2.39068194e-04,   2.40940622e-04,
         1.56242586e-04,   2.16980177e-04,   1.49836740e-06,
         1.28421611e-04,   4.93635298e-05,   1.12403468e-04,
         3.85117285e-06,   3.51473841e-05,   4.97705826e-05,
        -5.52613211e-03,   5.07535550e-04,   3.02473465e-04,
         2.85215840e-04,  -4.81568826e-03,  -4.06928334e-03,
        -2.40912130e-03,  -8.29316455e-04,   6.02690836e-04,
         1.61179754e-03,   1.97690761e-03,   1.83290810e-03,
         1.30492494e-03,   1.60267242e-04,   4.42708990e-03,
         7.45657605e-03,   6.44312976e-03,  -7.96582060e-04,
        -8.75941507e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 480
      nit: 6
   status: 0
  success: True
        x: array([ 1.        ,  0.99999911,  0.99999806,  0.99999706,  0.99999606,
        0.99999497,  0.99999388,  0.99999278,  0.99999163,  

In [6]:
ois_curve = DiscountCurve(today, pillar_dates, result.x)

In [7]:
from finmarkets import ForwardRateCurve

euribor_curves = {}
euribor_curves[1] = euribor_1m_curve
euribor_curves[3] = euribor_3m_curve
euribor_curves[6] = euribor_6m_curve

Create a BasisSwap class that has a method to compute the fair margin as follows:

$$S = \frac {S_{t2} - S_{t1}}{\text{annuity}_{t1}} $$

In [8]:
bs_mat = [bsw['maturity'] for bsw in basis_swaps]
bs_ft = [bsw['first_tenor'] for bsw in basis_swaps]
bs_st = [bsw['second_tenor'] for bsw in basis_swaps]

In [37]:
from datetime import date
from dateutil.relativedelta import relativedelta
from finmarkets import generate_swap_dates

class BasisRateSwap(object):
    def __init__(self, start_date, notional, 
                 maturity, first_tenor, second_tenor):
    
        self.start_date = today
        self.notional = notional
        self.maturity = maturity
        self.tenor1 = first_tenor
        self.tenor2 = second_tenor
    
        self.tenor1_payment_dates = \
            generate_swap_dates(self.start_date, 
                                self.maturity, 
                                self.tenor1)
        self.tenor2_payment_dates = \
            generate_swap_dates(self.start_date, 
                                self.maturity, 
                                self.tenor2)

    def swap_rate(self, pd, dc, euribor, corr=0):
        sp = 0 
        for i in range(1, len(pd)):
            F = euribor.forward_rate(pd[i-1]) + corr
            D = dc.df(pd[i])
            tau = (pd[i] - pd[i-1]).days / 360
            sp += F * D * tau
        return sp
        
    def annuity(self, payment_dates, dc):
        annuity_ft = 0
        for j in range(1, len(payment_dates)):
            D = dc.df(payment_dates[j])
            tau = (payment_dates[j] - payment_dates[j-1]).days/360
            annuity_ft += D * tau
        return annuity_ft
        
    def fair_margin(self, discount_curve, euribor_1t, euribor_2t):
        s2 = self.swap_rate(self.tenor2_payment_dates, discount_curve, euribor_2t)
        s1 = self.swap_rate(self.tenor1_payment_dates, discount_curve, euribor_1t) 
        delta_sp = s2 - s1
        
        annuity_ft = self.annuity(self.tenor1_payment_dates, discount_curve)
        return (delta_sp / annuity_ft)
    
    def fair_npv(self, discount_curve, euribor_1t, euribor_2t):
        par_spread = self.fair_margin(discount_curve, euribor_1t, euribor_2t)
        npv2 = self.swap_rate(self.tenor2_payment_dates, discount_curve, euribor_2t)
        npv1 = self.swap_rate(self.tenor1_payment_dates, discount_curve, euribor_1t, par_spread)
        
        return (npv1 - npv2)* self.notional
    
    def npv(self, discount_curve, euribor_1t, euribor_2t):
        par_spread = self.fair_margin(discount_curve, euribor_1t, euribor_2t)
        npv2 = self.swap_rate(self.tenor2_payment_dates, discount_curve, euribor_2t)
        npv1 = self.swap_rate(self.tenor1_payment_dates, discount_curve, euribor_1t)
        
        return (npv1 - npv2)* self.notional

In [38]:
par_spreads = []

for i in range(len(bs_mat)):
    par_spreads.append(BasisRateSwap(today, 1000000, bs_mat[i],
                       bs_ft[i], bs_st[i]).fair_margin(ois_curve, 
                                                       euribor_curves[bs_ft[i]],
                                                       euribor_curves[bs_st[i]]))

print ("Par spreads: " + str(["{:.5f}".format(s) for s in par_spreads]))

Par spreads: ['0.00030', '0.00129', '0.00159', '0.00008', '0.00127', '0.00135', '-0.00005', '0.00122', '0.00117', '-0.00008', '0.00117', '0.00110', '-0.00009', '0.00113', '0.00104']


In [39]:
bs = BasisRateSwap(today, 100, bs_mat[2], bs_ft[2], bs_st[2])

In [40]:
bs.fair_npv(ois_curve, euribor_curves[bs_ft[2]], euribor_curves[bs_st[2]])


8.673617379884035e-17

In [41]:
bs.npv(ois_curve, euribor_curves[bs_ft[2]], euribor_curves[bs_st[2]])

-0.16115462465650987